In [1]:
import os
import random
import pandas as pd
import numpy as np
import re
import json

import matplotlib.pyplot as plt
import seaborn as sns

from script.timestamp_smoothing import process_timestamp_correction,detect_jitter_patterns
from script.data_structures import BaseInfo, ACC_Raw_Data, WearInfo, PPG_Raw_Data, TYHX_PPG_Data
from script.data_types import get_full_data_type
from script.common import (
    get_raw_data, 
    parse_filename, 
    expand_acc_data, 
    expand_ppg_data, 
    merge_sensor_data,
    expand_sensor_data,
    process_and_merge_sensor_data
)
from script.wear_detection_dll import *

# 设置matplotlib参数
plt.style.use('default')  # 使用默认样式

# 尝试设置中文字体支持(如果失败则使用默认字体)
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
    plt.rcParams['axes.unicode_minus'] = False
except:
    print("注意: 中文字体设置失败,将使用默认字体")
    pass

In [2]:
def run_wear_detection_algorithm(df, directory,file_name_without_ext,
                                 ir_threshold=12000,green_threshold= 18000 ,ir_baseline=0
                                 ):
    # 1. 创建一个列表用于存储日志
    log_buffer = []

    # 2. 定义回调：只存列表,不写文件
    def memory_logging_callback(message):
        # print(message, end='') 
        log_buffer.append(message) # 存入内存
    
    dll_path = r"G:/ncs_workspace/application/algorithm_lib/alg_wear/build/bin/libalg_wear.dll"
    results = []
    try:
        # 使用上下文管理器自动管理DLL资源
        with WearDetectionDLL(dll_path, print_callback=memory_logging_callback) as dll:
            
            # 获取版本
            version = dll.get_version()
            print(f"alg_wear_new Ver: {version.major}.{version.minor}.{version.patch}")
            
            # 初始化(戒指模式)
            is_ring=True
            config = AlgWearInitParament()
            config.ppg_low_ir_threshold = ir_threshold
            config.ppg_green_threshold = green_threshold
            config.ppg_unwear_cnt = 10
            config.is_ring_flag = is_ring
            # config.wear_amb_std_threshold = 0
            # config.acc_quite_time = 300
            # config.green_confusion_threshold = 0
            # config.confirmed_recheck_timer = 60
            config.ppg_low_ir_baseline = ir_baseline
            dll.init_with_config(config)


            for idx, row in df.iterrows():
                
                
                # 1. 准备 ACC
                acc_data = row['acc_data']
                acc_axis = dll.prepare_acc_data(acc_data)
                unix_timestamp = row['unix_timestamp']
                # 2. 准备 PPG
                ppg_g = row['ppg_g_data']
                ppg_ir = row['ppg_ir_data']
                current = row['g_current']
                # print(f"Processing row {idx}", )
                # print(f"  ACC data: {acc_data}")
                # print(f"  PPG G data: {ppg_g}")
                # print(f"  PPG IR data: {ppg_ir}")

                # 构造 ppg_data 字典
                ppg_config = {
                    'g': ppg_g,
                    'low_ir': ppg_ir, # 使用 IR 数据
                    'channel_num': 1,
                    'g_current': current, # 模拟电流值
                    'fs': 32, 
                }
                ppg_sensors = dll.prepare_ppg_data(ppg_config)

                # 3. 准备 WearingData
                wearing_data = WearingData()
                wearing_data.measure_type = WearMeasureType.WEAR_IR

                # 4. 执行
                result = dll.process(acc_axis=acc_axis, ppg_sensors=ppg_sensors, wearing_data=wearing_data)
                result_dict = dll.result_to_dict(result)
                result_dict['unix_timestamp'] = unix_timestamp
                # 打印简单的结果验证
                # state_str = "WEAR" if result_dict.get('is_wear') else "UNWEAR" # 假设结果里有这个字段
                # print(f"-> Done. Result: {result_dict}")
                
                results.append(result_dict)

            print(f"\nSuccessfully processed {len(results)} rows.")

    
    except Exception as e:
        print(f"Error running wear detection: {e}")

    finally:
        # 4. 【核心】在 finally 块中将内存中的日志一次性写入文件
        # 这样无论是否报错,日志都会被保存
        log_file_path = os.path.join(directory, f"{file_name_without_ext}_log.txt")
        try:
            with open(log_file_path, 'w', encoding='utf-8') as f:
                # 将列表中的字符串拼接后写入,或者逐行写入
                # 注意：如果 message 自带换行符,就不用加 \n
                for msg in log_buffer:
                    f.write(msg) 
            print(f"日志已保存至: {log_file_path}")
        except Exception as io_e:
            print(f"保存日志失败: {io_e}")

            
    return results



In [3]:
def plot_wear_detection_results(expanded_ppg_ir_df, expanded_ppg_g_df, expanded_ppg_r_df, expanded_acc_df, wear_df, directory, file_name_without_ext, is_show=False, result_type='original'):
    """
    绘制佩戴检测结果数据,包括展开的PPG IR、PPG Green、PPG Red、加速度数据和佩戴标志。
    根据可用数据动态调整子图数量和布局。
    """
    # 检查每个数据源是否有效
    has_ppg_ir = expanded_ppg_ir_df is not None and not expanded_ppg_ir_df.empty
    has_ppg_g = expanded_ppg_g_df is not None and not expanded_ppg_g_df.empty
    has_ppg_r = expanded_ppg_r_df is not None and not expanded_ppg_r_df.empty
    has_acc = expanded_acc_df is not None and not expanded_acc_df.empty
    has_wear = wear_df is not None and not wear_df.empty

    # 计算需要的子图数量(PPG图作为一个子图）
    plot_count = sum([has_ppg_ir or has_ppg_g or has_ppg_r, has_acc, has_wear])

    if plot_count == 0:
        print("没有可用的数据进行绘制")
        return

    fig, axes = plt.subplots(plot_count, 1, figsize=(20, 4 * plot_count), sharex=True)
    if plot_count == 1:
        axes = [axes]

    result_label = "Original" if result_type == 'original' else "Simulated"
    fig.suptitle(f"{file_name_without_ext} - {result_label}", fontsize=16, fontweight='bold')
    current_ax_idx = 0

    # 绘制 PPG 数据(IR / Green / Red）
    if has_ppg_ir or has_ppg_g or has_ppg_r:
        ax = axes[current_ax_idx]

        # 为了清晰,使用双 Y 轴：左轴绘制 IR 和 Red,右轴绘制 Green(如果存在）
        ax_twinx = None
        if has_ppg_ir:
            ax.plot(expanded_ppg_ir_df['unix_timestamp'], expanded_ppg_ir_df['ppg_value'], label='PPG IR', color='pink', linewidth=1)
        if has_ppg_r:
            ax.plot(expanded_ppg_r_df['unix_timestamp'], expanded_ppg_r_df['ppg_value'], '.',markersize=1, label='PPG Red', color='red', linewidth=1)
        if has_ppg_g:
            ax_twinx = ax.twinx()
            ax_twinx.set_ylabel('PPG Green', fontsize=12)
            # ax_twinx.plot(expanded_ppg_g_df['unix_timestamp'], expanded_ppg_g_df['ppg_value'], '.',markersize=1, label='PPG Green', color='green', linewidth=0.5) # type: ignore
            ax_twinx.plot(expanded_ppg_g_df['unix_timestamp'], expanded_ppg_g_df['ppg_value'], label='PPG Green', color='green', linewidth=0.5) # type: ignore

        ax.set_title('PPG Data')
        ax.set_ylabel('PPG (IR/Red)')
        ax.grid(True, alpha=0.3)

        # 合并图例
        handles, labels = ax.get_legend_handles_labels()
        if ax_twinx is not None:
            h2, l2 = ax_twinx.get_legend_handles_labels()
            handles += h2
            labels += l2
        if handles:
            ax.legend(handles, labels, loc='upper left')

        current_ax_idx += 1

    # 绘制加速度数据
    if has_acc:
        ax = axes[current_ax_idx]
        ax.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['x'], label='ACC X', color='red', linewidth=0.5)
        ax.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['y'], label='ACC Y', color='green', linewidth=0.5)
        ax.plot(expanded_acc_df['unix_timestamp'], expanded_acc_df['z'], label='ACC Z', color='blue', linewidth=0.5)
        ax.set_ylabel('Acceleration')
        ax.set_title('Acceleration Data')
        ax.legend()
        ax.grid(True, alpha=0.3)
        current_ax_idx += 1

    # 绘制佩戴标志
    if has_wear:
        ax = axes[current_ax_idx]
        ax.plot(wear_df['unix_timestamp'], wear_df['wear_flag'], marker='o', markersize=3, linewidth=1, label='Wear Flag', color='orange')
        ax.set_ylabel('Wear Flag')
        ax.set_xlabel('Unix Timestamp (s)')
        ax.set_title('Wear Info Flags')
        ax.legend()
        ax.grid(True, alpha=0.3)

    suffix = "_original" if result_type == 'original' else "_simulated"
    output_file = f"{directory}/{file_name_without_ext}_wear{suffix}.png"

    plt.savefig(output_file, dpi=300)
    print(f"图形已保存至: {output_file}")
    plt.tight_layout()
    if is_show:
        plt.show()
    else:
        plt.close()


In [4]:
def plot_single_ppg(df, title='PPG Signal', color='blue'):
    """
    绘制单个PPG数据的时间序列图。
    
    参数:
        df: DataFrame,必须包含 'unix_timestamp' 和 'ppg_value' 列
        directory: 保存目录
        file_name_without_ext: 文件名(不含扩展名)
        title: 图形标题
        color: 曲线颜色
    """
    # 检查数据是否有效
    if df is None or df.empty:
        print("没有可用的数据进行绘制")
        return
    
    # 检查必需的列是否存在
    if 'unix_timestamp' not in df.columns or 'ppg_value' not in df.columns:
        print("错误:DataFrame必须包含 'unix_timestamp' 和 'ppg_value' 列")
        return
    
    # 创建图形
    fig, ax = plt.subplots(figsize=(20, 6))
    
    # 绘制数据
    ax.plot(df['unix_timestamp'], df['ppg_value'], 
            color=color, linewidth=0.8, label='PPG Signal')
    
    # 设置标签和标题
    ax.set_xlabel('Unix Timestamp (s)', fontsize=12)
    ax.set_ylabel('PPG Value', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    
    # 添加网格和图例
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left')
    plt.tight_layout()

    plt.show()


In [5]:
def plot_multi_ppg_data(ppg_g_df=None,
                        ppg_g2_df=None,
                        ppg_ir_df=None,
                        ppg_r_df=None,
                        ppg_r_ir_df=None,
                        acc_df=None,
                        directory='.',
                        file_name_without_ext='output',
                        is_show=False):
    """
    绘制多通道PPG数据和加速度数据。
    子图数量由数据存在性决定,顺序固定且不变：
      1) PPG Green 或 PPG IR
      2) PPG Red 或 PPG R-IR
      3) 加速度(若存在则绘制合加速度或三轴)
    画布大小固定为 (20, 12)。

    参数:
        ppg_g_df, ppg_ir_df, ppg_r_df, ppg_r_ir_df: DataFrame or None
            每个 DataFrame 应包含列 `unix_timestamp` 和 `ppg_value`(若存在)
        acc_df: 加速度 DataFrame(若存在),优先查找 ['x','y','z'] 三轴并绘制合加速度
        directory: 保存目录
        file_name_without_ext: 输出文件名(不含扩展名)
        is_show: 是否在绘制后显示窗口
    """
    import os
    import numpy as np
    import matplotlib.pyplot as plt

    # 数据存在性判断
    has_ppg_g = ppg_g_df is not None and not ppg_g_df.empty
    has_ppg_g2 = ppg_g2_df is not None and not ppg_g2_df.empty
    has_ppg_ir = ppg_ir_df is not None and not ppg_ir_df.empty
    has_ppg_r = ppg_r_df is not None and not ppg_r_df.empty
    has_ppg_r_ir = ppg_r_ir_df is not None and not ppg_r_ir_df.empty
    has_acc = acc_df is not None and not acc_df.empty

    # 逻辑分组(保持顺序不变)
    group1 = has_ppg_g or has_ppg_ir
    group2 = has_ppg_r or has_ppg_r_ir
    group3 = has_acc

    plot_count = sum([group1, group2, group3])
    if plot_count == 0:
        print("没有可用的数据进行绘制")
        return

    # 固定画布大小 (20,12),子图数由数据决定
    fig, axes = plt.subplots(plot_count, 1, figsize=(20, 12), sharex=True)
    if plot_count == 1:
        axes = [axes]
    fig.suptitle(f"{file_name_without_ext} - Multi-PPG Analysis", fontsize=16, fontweight='bold')

    idx = 0

    # group1: PPG Green (左轴) + PPG IR (右轴)
    if group1:
        ax = axes[idx]
        any_drawn = False
        ax_twin = None
        # 绘制第一条线 (PPG Green)
        if has_ppg_g:
            if 'unix_timestamp' in ppg_g_df.columns and 'ppg_value' in ppg_g_df.columns:
                ax.plot(ppg_g_df['unix_timestamp'], ppg_g_df['ppg_value'],
                        label='PPG Green', color='green', linewidth=0.8)
                ax.set_ylabel('PPG Green', fontsize=12, color='green')
                ax.tick_params(axis='y', labelcolor='green')
                any_drawn = True

        # 绘制第二条线 (PPG2 Green / Orange)
        if has_ppg_g2:
            if 'unix_timestamp' in ppg_g2_df.columns and 'ppg_value' in ppg_g2_df.columns:
                ax.plot(ppg_g2_df['unix_timestamp'], ppg_g2_df['ppg_value'],
                        label='PPG2 Green', color='purple', linewidth=0.8)
                
                # 如果只有第二条线没有第一条线,可能需要补设 Label (可选优化)
                if not has_ppg_g: 
                     ax.set_ylabel('PPG2 Green', fontsize=12, color='purple')
                     ax.tick_params(axis='y', labelcolor='purple')
                any_drawn = True

        if has_ppg_ir:
            ax_twin = ax.twinx()
            if 'unix_timestamp' in ppg_ir_df.columns and 'ppg_value' in ppg_ir_df.columns:
                ax_twin.plot(ppg_ir_df['unix_timestamp'], ppg_ir_df['ppg_value'],
                             label='PPG IR', color='pink', linewidth=0.8)
                ax_twin.set_ylabel('PPG IR', fontsize=12, color='pink')
                ax_twin.tick_params(axis='y', labelcolor='pink')
                any_drawn = True
        ax.set_title('PPG Green & IR', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        # 合并图例
        handles, labels = ax.get_legend_handles_labels()
        if ax_twin is not None:
            h2, l2 = ax_twin.get_legend_handles_labels()
            handles += h2
            labels += l2
        if handles:
            ax.legend(handles, labels, loc='upper left')
        if not any_drawn:
            ax.text(0.5, 0.5, 'No Data Available', ha='center', va='center',
                    transform=ax.transAxes, fontsize=14, color='gray')
        idx += 1

    # group2: PPG Red (左轴) + PPG R-IR (右轴)
    if group2:
        ax = axes[idx]
        any_drawn = False
        ax_twin = None
        if has_ppg_r:
            if 'unix_timestamp' in ppg_r_df.columns and 'ppg_value' in ppg_r_df.columns:
                ax.plot(ppg_r_df['unix_timestamp'], ppg_r_df['ppg_value'],
                        label='PPG Red', color='red', linewidth=0.8)
                ax.set_ylabel('PPG Red', fontsize=12, color='red')
                ax.tick_params(axis='y', labelcolor='red')
                any_drawn = True
        if has_ppg_r_ir:
            ax_twin = ax.twinx()
            if 'unix_timestamp' in ppg_r_ir_df.columns and 'ppg_value' in ppg_r_ir_df.columns:
                ax_twin.plot(ppg_r_ir_df['unix_timestamp'], ppg_r_ir_df['ppg_value'],
                             label='PPG R-IR', color='purple', linewidth=0.8)
                ax_twin.set_ylabel('PPG R-IR', fontsize=12, color='purple')
                ax_twin.tick_params(axis='y', labelcolor='purple')
                any_drawn = True
        ax.set_title('PPG Red & R-IR', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        handles, labels = ax.get_legend_handles_labels()
        if ax_twin is not None:
            h2, l2 = ax_twin.get_legend_handles_labels()
            handles += h2
            labels += l2
        if handles:
            ax.legend(handles, labels, loc='upper left')
        if not any_drawn:
            ax.text(0.5, 0.5, 'No Data Available', ha='center', va='center',
                    transform=ax.transAxes, fontsize=14, color='gray')
        idx += 1

    # group3: Acceleration - 绘制合加速度(优先)或三轴
    if group3:
        ax = axes[idx]
        if has_acc:
            # 优先查找 x,y,z 三轴,计算合加速度并绘制；否则尝试常见合加速度字段
            if all(c in acc_df.columns for c in ['x', 'y', 'z']):
                ax.plot(acc_df['unix_timestamp'], acc_df['x'], label='ACC X', color='red', linewidth=0.5)
                ax.plot(acc_df['unix_timestamp'], acc_df['y'], label='ACC Y', color='green', linewidth=0.5)
                ax.plot(acc_df['unix_timestamp'], acc_df['z'], label='ACC Z', color='blue', linewidth=0.5)
                # 计算合加速度并绘制(半透明)
                mag = np.sqrt(acc_df['x'].astype(float)**2 + acc_df['y'].astype(float)**2 + acc_df['z'].astype(float)**2)
                ax.plot(acc_df['unix_timestamp'], mag, label='ACC Magnitude', color='black', linewidth=0.8, alpha=0.6)
            else:
                # 查找可能代表合加速度的列名
                possible = [col for col in acc_df.columns if col.lower() in ('acc', 'acc_norm', 'magnitude', 'mag', 'a', 'acc_mag')]
                if possible:
                    ax.plot(acc_df['unix_timestamp'], acc_df[possible[0]], label=possible[0], color='purple', linewidth=0.8)
                else:
                    ax.text(0.5, 0.5, 'No Acceleration Columns Found', ha='center', va='center',
                            transform=ax.transAxes, fontsize=12, color='gray')
            ax.set_ylabel('Acceleration', fontsize=12)
            ax.legend(loc='upper left')
        else:
            ax.set_ylabel('Acceleration (No Data)', fontsize=12, color='gray')
            ax.text(0.5, 0.5, 'No Data Available', ha='center', va='center',
                    transform=ax.transAxes, fontsize=14, color='gray')
        ax.set_title('Acceleration Data', fontsize=14, fontweight='bold')
        ax.set_xlabel('Unix Timestamp (s)', fontsize=12)
        ax.grid(True, alpha=0.3)
        idx += 1

    # 保存并显示/关闭
    os.makedirs(directory, exist_ok=True)
    output_file = os.path.join(directory, f"{file_name_without_ext}_multi_ppg.png")
    plt.tight_layout()
    plt.savefig(output_file, dpi=300)
    print(f"图形已保存至: {output_file}")
    if is_show:
        plt.show()
    else:
        plt.close()

In [6]:
def plot_tyhx_currents(tyhx_df, directory='.', file_name_without_ext='tyhx', is_show=False, figsize=(20, 12)):
    """
    在 2x2 子图中绘制 tyhx_df 中 unix_timestamp 与指定电流与 offset 列的关系。
    布局 (row, col):
      (0,0) Green      -> left: green_current       right: green_offset_idac
      (0,1) Green2     -> left: green2_current      right: green2_offset_idac
      (1,0) Red        -> left: red_current         right: red_offset_idac
      (1,1) Infra      -> left: infra_current       right: infra_offset_idac

    左轴绘制 current,右轴绘制 offset_idac。
    """

    if tyhx_df is None or tyhx_df.empty:
        print("tyhx_df 为空,无法绘制。")
        return

    os.makedirs(directory, exist_ok=True)

    pairs = [
        ('green_current', 'green_offset_idac', 'Green'),
        ('green2_current', 'green2_offset_idac', 'Green2'),
        ('red_current', 'red_offset_idac', 'Red'),
        ('infra_current', 'infra_offset_idac', 'Infra'),
    ]

    fig, axes = plt.subplots(2, 2, figsize=figsize, sharex=True)
    axes = axes.flatten()

    for i, (curr_col, off_col, title) in enumerate(pairs):
        ax = axes[i]
        ax_right = None
        drawn_left = False
        drawn_right = False

        # 绘制左轴（current）
        if curr_col in tyhx_df.columns and not tyhx_df[curr_col].dropna().empty:
            ax.plot(tyhx_df['unix_timestamp'], tyhx_df[curr_col],
                    label=curr_col, color='tab:blue', linewidth=0.8)
            ax.set_ylabel('Current', fontsize=10, color='tab:blue')
            ax.tick_params(axis='y', labelcolor='tab:blue')
            drawn_left = True

        # 绘制右轴（offset idac）
        if off_col in tyhx_df.columns and not tyhx_df[off_col].dropna().empty:
            ax_right = ax.twinx()
            ax_right.plot(tyhx_df['unix_timestamp'], tyhx_df[off_col],
                          label=off_col, color='tab:orange', linewidth=0.8)
            ax_right.set_ylabel('Offset (IDAC)', fontsize=10, color='tab:orange')
            ax_right.tick_params(axis='y', labelcolor='tab:orange')
            ax_right.set_ylim(-1, 8)
            drawn_right = True

        ax.set_title(f"{title} ({curr_col} / {off_col})", fontsize=12, fontweight='bold')
        ax.grid(True, alpha=0.3)

        # 合并图例：左轴图例放左上,右轴图例合并到左上
        handles, labels = ax.get_legend_handles_labels()
        if ax_right is not None:
            h2, l2 = ax_right.get_legend_handles_labels()
            handles += h2
            labels += l2
        if handles:
            ax.legend(handles, labels, loc='upper left')

        if not (drawn_left or drawn_right):
            ax.text(0.5, 0.5, 'No Data', ha='center', va='center', transform=ax.transAxes, color='gray')

    # 统一 x 标签放在底部中间子图
    axes[2].set_xlabel('Unix Timestamp (s)', fontsize=11)

    plt.tight_layout()
    output_file = os.path.join(directory, f"{file_name_without_ext}_tyhx_currents.png")
    plt.savefig(output_file, dpi=300)
    print(f"图形已保存至: {output_file}")
    if is_show:
        plt.show()
    else:
        plt.close()

In [ ]:
# # %matplotlib qt
# %matplotlib inline  



# directory = r'rawdata/Wear/0108'

# # 获取目录下的所有文件
# all_files = os.listdir(directory)

# # 过滤出 .rawdata 文件
# rawdata_files = [f for f in all_files if f.endswith('.rawdata')]

# # 随机打乱文件顺序
# # random.shuffle(rawdata_files)


# 




# df_dict = {}
# raw_df_dict = {}
# for f in rawdata_files:
#     file_path = os.path.join(directory, f)

#     real_path = os.path.realpath(file_path)
#     file_name = os.path.basename(file_path)
#     file_name_without_ext, ext = os.path.splitext(file_name)
#     directory = os.path.dirname(real_path)
#     info =  parse_filename(file_name)
#     mac = info.get('mac','0000').lower()
    
#     print(f"real_path: {real_path} {directory} {file_name_without_ext}{ext}  {mac} \n")

#     result = get_raw_data(real_path)


#     base_info_df = result.get('base_info_df', None)
#     ppg_g_df = result.get('ppg_g_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
#     ppg_ir_df = result.get('ppg_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
#     ppg_r_df = result.get('ppg_r_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
#     wear_info_df = result.get('wear_info_df', None)
#     acc_df =  result.get('acc_df', None)
#     data_type_dict = result.get('data_types', {})

#     raw_df_dict[file_name_without_ext] = (
#         base_info_df,
#         ppg_g_df,
#         ppg_ir_df,
#         wear_info_df,
#         acc_df
#     )


#     sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
#     readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 


#     print(f"文件 {file_path} 解析完成,包含数据类型: ")
#     print(f"{"\\n".join(readable_types)}")

#     (combined_ppg_df, 
#     wear_info_df,
#     expanded_acc_df,
#     expanded_ppg_g_df,
#     expanded_ppg_r_df,
#     expanded_ppg_ir_df)= process_and_merge_sensor_data(acc_df, ppg_g_df, ppg_ir_df,ppg_r_df, wear_info_df) 
        
#     df_dict[file_name_without_ext] = (combined_ppg_df, 
#     wear_info_df,
#     expanded_acc_df,
#     expanded_ppg_g_df,
#     expanded_ppg_ir_df,
#     expanded_ppg_r_df)
#     # 调用函数绘制结果
#     plot_wear_detection_results(expanded_ppg_ir_df, expanded_ppg_g_df, expanded_ppg_r_df, 
#                                 expanded_acc_df, wear_info_df, directory, 
#                                 file_name_without_ext, is_show=False,result_type='original')
#     # wear_info_df.to_csv(f"{directory}/{file_name_without_ext}_wear_info.csv", index=False, encoding='utf-8')



#     ir_baseline = int(mac_baseline.get(mac,0))
#     print(f"使用的ir_baseline: {ir_baseline} for mac: {mac}")
#     wear_result = run_wear_detection_algorithm(combined_ppg_df,directory,file_name_without_ext,
#                                                 ir_threshold=20000,
#                                                 ir_baseline=ir_baseline)
#     wear_df = pd.DataFrame(wear_result)
#     if wear_df is not None and not wear_df.empty:
#         plot_wear_detection_results(expanded_ppg_ir_df, expanded_ppg_g_df, expanded_ppg_r_df, 
#                                 expanded_acc_df, wear_df, directory, 
#                                 file_name_without_ext, is_show=False,result_type='simulated')

In [7]:
def resample_to_25hz(df, value_columns, data_name="data"):
    """
    将数据降采样到25Hz
    
    参数:
        df: 包含unix_timestamp和数据列的DataFrame
        value_columns: 需要保留的数值列名列表
        data_name: 数据名称(用于打印信息)
    
    返回:
        降采样后的DataFrame
    """
    if df is None or df.empty:
        print(f"{data_name} 为空或不存在")
        return None
    
    # 创建副本避免修改原始数据
    resampled = df.copy()
    
    # 将unix_timestamp转换为datetime并设置为索引
    resampled['datetime'] = pd.to_datetime(resampled['unix_timestamp'], unit='s')
    resampled.set_index('datetime', inplace=True)
    
    # 降采样到25Hz (即每40ms一个点)
    target_freq = '40ms'  # 1000ms / 25Hz = 40ms
    resampled_25hz = resampled.resample(target_freq).mean()
    
    # 重置索引并重新计算unix_timestamp
    resampled_25hz.reset_index(inplace=True)
    resampled_25hz['unix_timestamp'] = resampled_25hz['datetime'].astype(np.int64) / 1e9
    
    # 删除datetime列,只保留需要的列
    columns_to_keep = ['unix_timestamp'] + value_columns
    resampled_25hz = resampled_25hz[columns_to_keep].dropna()
    
    # 分析降采样后的数据
    start_time = resampled_25hz['unix_timestamp'].min()
    end_time = resampled_25hz['unix_timestamp'].max()
    total_rows = len(resampled_25hz)
    duration = end_time - start_time
    
    print(f"{data_name} 降采样结果:")
    print(f"  起始时间: {start_time:.3f}, 结束时间: {end_time:.3f}")
    print(f"  持续时间: {duration:.2f}秒 ({duration/60:.2f}分钟)")
    print(f"  数据行数: {total_rows}, 采样率: {total_rows/duration:.2f} Hz" if duration > 0 else "  无法计算采样率")
    print(f"  原始数据: {len(df)}行 → 降采样后: {total_rows}行 (减少{(1-total_rows/len(df))*100:.2f}%)")
    
    return resampled_25hz


def merge_acc_ppg_data(acc_df, ppg_df, tolerance_ms=20):
    """
    合并加速度和PPG数据(基于时间戳的最近匹配)
    
    参数:
        acc_df: 加速度DataFrame (包含unix_timestamp, x, y, z)
        ppg_df: PPG DataFrame (包含unix_timestamp, ppg_value)
        tolerance_ms: 时间容差(毫秒)
    
    返回:
        合并后的DataFrame
    """
    if acc_df is None or acc_df.empty or ppg_df is None or ppg_df.empty:
        print("ACC或PPG数据为空,无法合并")
        return pd.DataFrame()
    
    print("\n开始合并ACC和PPG数据:")
    print(f"  ACC数据: {len(acc_df)}行")
    print(f"  PPG数据: {len(ppg_df)}行")
    print(f"  时间容差: {tolerance_ms}ms")
    
    # 创建临时的datetime列用于合并
    ppg_temp = ppg_df.copy()
    acc_temp = acc_df.copy()
    ppg_temp['merge_time'] = pd.to_datetime(ppg_temp['unix_timestamp'], unit='s')
    acc_temp['merge_time'] = pd.to_datetime(acc_temp['unix_timestamp'], unit='s')
    
    # 使用merge_asof进行最近时间点合并
    merged_data = pd.merge_asof(
        ppg_temp.sort_values('merge_time'),
        acc_temp.sort_values('merge_time'),
        on='merge_time',
        direction='nearest',
        tolerance=pd.Timedelta(milliseconds=tolerance_ms)
    )
    
    # 删除临时列,保留原始的unix_timestamp
    merged_data = merged_data.drop(columns=['merge_time', 'unix_timestamp_y'])
    merged_data = merged_data.rename(columns={'unix_timestamp_x': 'unix_timestamp'})
    
    # 删除没有匹配的行
    merged_data = merged_data.dropna()
    

    print(f"\n合并完成:")
    print(f"  合并后行数: {len(merged_data)}")
    print(f"  成功率: ACC {len(merged_data)/len(acc_df)*100:.2f}%, PPG {len(merged_data)/len(ppg_df)*100:.2f}%")
    print(f"  数据列: {merged_data.columns.tolist()}")
    
    return merged_data

In [8]:
%matplotlib qt
# %matplotlib inline
is_ring = False

#                                               类型  电流  offset
# HeartRate_Static_TEST_20260124_112928_de54    日常  74   9
# HeartRate_Static_TEST_20260124_114011_de54    运动  60   9 
# HeartRate_Static_TEST_20260124_115524_de54    血氧  142  9 

# directory = r'G:\PyScript\rawdata\Wear\BUG'
# file_name = "HeartRate_Static_xf_20260122_162945_afc2.rawdata"


# # Sleep_Normal_hlh_20260126_224358_f246
# directory = r'G:\PyScript\rawdata\Wear\CHR03'
# file_name = "Sleep_Normal_hlh_20260126_224358_f246.rawdata"


is_ring = True

file_path = os.path.join(directory, file_name)

real_path = os.path.realpath(file_path)
file_name = os.path.basename(file_path)
file_name_without_ext, ext = os.path.splitext(file_name)
directory = os.path.dirname(real_path)
info =  parse_filename(file_name)
mac = info.get('mac','0000').lower()

print(f"real_path: {real_path} {directory} {file_name_without_ext}{ext}  {mac} \n")

result = get_raw_data(real_path,is_ring=is_ring)

save_dir = os.path.join(directory, "data")
os.makedirs(save_dir, exist_ok=True)


data_type_dict = result.get('data_types', {})

base_info_df = result.get('base_info_df', pd.DataFrame(columns=['unix_timestamp']))
wear_info_df = result.get('wear_info_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_df = result.get('TYHX_data_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_agc_data_df = result.get('TYHX_agc_data_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_3697_data_df = result.get('TYHX_3697_data_df', pd.DataFrame(columns=['unix_timestamp']))
psp_fifo_df = result.get('psp_fifo_df', pd.DataFrame(columns=['unix_timestamp']))

ppg_g_df = result.get('ppg_g_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_r_df = result.get('ppg_r_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_ir_df = result.get('ppg_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_r_ir_df = result.get('ppg_r_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
acc_df =  result.get('acc_df', pd.DataFrame(columns=['unix_timestamp']))


sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 


print(f"文件 {file_path} 解析完成,包含数据类型: ")
print(f"{"\n".join(readable_types)}")

(combined_ppg_df, 
wear_info_df,
expanded_acc_df, 
expanded_ppg_g_df, 
expanded_ppg_ir_df, 
expanded_ppg_r_df)= process_and_merge_sensor_data(acc_df, ppg_g_df, ppg_ir_df,ppg_r_df, wear_info_df) 

expanded_ppg_r_ir_df = expand_ppg_data(ppg_r_ir_df, 'ppg_data')
expanded_ppg_r_ir_df = expanded_ppg_r_ir_df.rename(columns={'ppg_data':'ppg_value'})

tyhx_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_data.csv", index=False, encoding='utf-8')
tyhx_agc_data_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_agc_data.csv", index=False, encoding='utf-8')
acc_df.to_csv(f"{save_dir}/{file_name_without_ext}_acc_data.csv", index=False, encoding='utf-8')
expanded_ppg_g_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_g.csv", index=False, encoding='utf-8')
expanded_ppg_r_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r.csv", index=False, encoding='utf-8')
expanded_ppg_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_ir.csv", index=False, encoding='utf-8')
expanded_ppg_r_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r_ir.csv", index=False, encoding='utf-8')
# combined_ppg_df.to_csv(f"{save_dir}/{file_name_without_ext}_combined_ppg.csv", index=False, encoding='utf-8')


# null_df = pd.DataFrame()




# 调用函数绘制结果
plot_wear_detection_results(expanded_ppg_ir_df, 
                            expanded_ppg_g_df, 
                            expanded_ppg_r_df, 
                            expanded_acc_df, 
                            wear_info_df, 
                            save_dir, 
                            file_name_without_ext,
                            is_show=True,result_type='original')

real_path: G:\PyScript\rawdata\Wear\CHR03\Sleep_Normal_hlh_20260126_224358_f246.rawdata G:\PyScript\rawdata\Wear\CHR03 Sleep_Normal_hlh_20260126_224358_f246.rawdata  f246 

文件 G:\PyScript\rawdata\Wear\CHR03\Sleep_Normal_hlh_20260126_224358_f246.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 33421 
15  RAW_PPG_GREEN         原始PPG绿光                count: 20586 
16  RAW_PPG_RED           原始PPG红光                count: 970 
17  RAW_PPG_IR            原始PPG红外                count: 9589 
18  RAW_ACC               原始加速度                  count: 66657 
21  RAW_ACC_TO_PPG        原始加速度到PPG              count: 64149 
26  RAW_PPG_RED_IR        原始PPG红灯红外              count: 835 
33  AMBIENT_INFO          环境信息                   count: 15607 
52  UNKNOWN(52)           未知类型(52)               count: 1 
67  SLEEP_INFO            睡眠信息                   count: 32555 
68  HRV_INFO              心率变异性信息                count: 31769 
69  MOTION_INFO           运动识别                   

g:\PyScript\.venv\Lib\site-packages\ipykernel\eventloops.py:158: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  el.exec() if hasattr(el, "exec") else el.exec_()


In [ ]:
df.to_clipboard()

In [9]:
%matplotlib qt
# %matplotlib inline

is_ring = False

# HeartRate_Static_TEST_20260126_155030_de54.rawdata  日常  
# HeartRate_Static_TEST_20260126_151852_de54.rawdata  血氧


# HeartRate_Static_TEST_20260126_155913_de54.rawdata
# HeartRate_Static_TEST_20260126_163313_de54.rawdata 运动


# HeartRate_Static_TEST_20260126_174005_de54.rawdata 日常
# HeartRate_Static_TEST_20260126_174143_de54.rawdata 血氧
# HeartRate_Static_TEST_20260126_174340_de54.rawdata 运动
#


#  HeartRate_Static_TEST_20260126_175701_de54.rawdata  运动
#  HeartRate_Static_TEST_20260126_175843_de54.rawdata  日常
#  HeartRate_Static_TEST_20260126_180247_de54.rawdata   血氧
# HeartRate_Static_TEST_20260126_181441_de54.rawdata
# HeartRate_Static_TEST_20260126_181712_de54.rawdata
# HeartRate_Static_TEST_20260126_192850_de54.rawdata 日常

# HeartRate_Static_TEST_20260126_193800_de54.rawdata 运动
# HeartRate_Static_TEST_20260126_193928_de54.rawdata 血氧
# HeartRate_Static_TEST_20260126_194109_de54.rawdata 日常

# HeartRate_Static_TEST_20260126_194656_de54.rawdata 运动
# HeartRate_Static_TEST_20260126_194819_de54.rawdata 血氧
# HeartRate_Static_TEST_20260126_195004_de54.rawdata 日常

# HeartRate_Static_TEST_20260126_195137_de54.rawdata 运动
# HeartRate_Static_TEST_20260126_195255_de54.rawdata 血氧
# HeartRate_Static_TEST_20260126_195430_de54.rawdata 日常


# HeartRate_Static_TEST_20260126_204313_de54.rawdata 运动
# HeartRate_Static_TEST_20260126_204517_de54.rawdata 血氧
# HeartRate_Static_TEST_20260126_204654_de54.rawdata 日常


# HeartRate_Static_TEST_20260127_102228_de54.rawdata
# HeartRate_Static_TEST_20260127_102348_de54.rawdata
# HeartRate_Static_TEST_20260127_102545_de54.rawdata
# HeartRate_Static_TEST_20260127_102738_de54.rawdata

# HeartRate_Static_TEST_20260127_120154_de54.rawdata
# HeartRate_Static_TEST_20260127_120335_de54.rawdata
# HeartRate_Static_TEST_20260127_120549_de54.rawdata

# HeartRate_Static_TEST_20260127_120906_de54.rawdata
# HeartRate_Static_TEST_20260127_121022_de54.rawdata
# HeartRate_Static_TEST_20260127_121204_de54.rawdata


# HeartRate_Static_TEST_20260127_150344_dc68.rawdata
# HeartRate_Static_TEST_20260127_150344_de54.rawdata
# HeartRate_Static_TEST_20260127_150529_dc68.rawdata
# HeartRate_Static_TEST_20260127_150529_de54.rawdata
# HeartRate_Static_TEST_20260127_150714_dc68.rawdata
# HeartRate_Static_TEST_20260127_150714_de54.rawdata
# HeartRate_Static_TEST_20260127_150901_dc68.rawdata
# HeartRate_Static_TEST_20260127_150901_de54.rawdata



# HeartRate_Static_TEST_20260127_151210_dc68.rawdata
# HeartRate_Static_TEST_20260127_151210_de54.rawdata
# HeartRate_Static_TEST_20260127_151359_dc68.rawdata
# HeartRate_Static_TEST_20260127_151359_de54.rawdata
# HeartRate_Static_TEST_20260127_151739_dc68.rawdata
# HeartRate_Static_TEST_20260127_151739_de54.rawdata
# HeartRate_Static_TEST_20260127_151926_dc68.rawdata
# HeartRate_Static_TEST_20260127_151926_de54.rawdata




# Customer Testing
# HeartRate_Static_sowjanya_20260127_194424_5e7a.rawdata First file with 4.05 MB - Yoga (3min)
# HeartRate_Static_sowjanya_20260127_194926_5e7a.rawdata Second file with 3.82 MB - Stable HR (3min)
# HeartRate_Static_sowjanya_20260127_195319_5e7a.rawdata One click measurement - able to measure both the times
# HeartRate_Static_sowjanya_20260127_195506_5e7a.rawdata

# HeartRate_Static_murali_20260127_195747_5e7a.rawdata Yoga
# HeartRate_Static_murali_20260127_200119_5e7a.rawdata Stable
# HeartRate_Static_murali_20260127_200439_5e7a.rawdata One click  able to measure
# HeartRate_Static_murali_20260127_200621_5e7a.rawdata One click  not able to measure
# HeartRate_Static_murali_20260127_200804_5e7a.rawdata One click  not able to measure
# HeartRate_Static_murali_20260127_200950_5e7a.rawdata One click  able to measure
# Note- The watch was worn slightly near to bone for first measurements(third, forth and fifth), 
# . Worn one finger back from bone for last measurement(sixth)

# Sleep_Normal_Jeven_20260128_022928_4d02  sleep 


# HeartRate_Static_Sagar_20260127_215637_b27b.rawdata  Yoga  
# HeartRate_Static_Sagar_20260127_220030_b27b.rawdata  Stable HR
# HeartRate_Static_Sagar_20260127_220452_b27b.rawdata  One click measurement - able to measure both the times

# Sleep_Normal_Pravin_20260127_222435_f859   End time is late by 1 hourWoke up at 6:04 am but sleep recorded till 7:04 am
# Sleep_Normal_murali_20260127_213934_dc20   This is log for user with awake time - was awake in between for 4 to 5 times (for around 1-2min everytime)

# Sleep_Normal_MuraliM_20260127_230140_5e7a   This is a big problem in sleep..Nap is recorded when kept on bed just for 20 min and went for shower. And why is it showing 0 for all focus, heart rate and HRV?




# directory = r'G:\PyScript\rawdata\Wear\CWR12G_0128_test'
# file_name = "Sleep_Normal_murali_20260127_213934_dc20.rawdata"




# HeartRate_Static_TEST_20260128_180423_f246
# HeartRate_Static_TEST_20260128_180531_f246

# HeartRate_IndoorWalk_cy_20260127_160122_44ee.rawdata
# HeartRate_IndoorWalk_ly_20260128_170347_aded.rawdata
# HeartRate_IndoorWalk_tdy_20260128_163127_aded.rawdata
# HeartRate_OutdoorWalk_tdy_20260128_161156_0395.rawdata
# HeartRate_OutdoorWalk_tdy_20260128_161156_aded


# HeartRate_Static_hlh_20260127_212503_3d64  戒指放置在办公桌面上未佩戴 佩戴异常
# HeartRate_Static_hlh_20260128_000150_f246


# HeartRate_Static_TEST_20260129_173816_e7af.rawdata
directory = r'G:\PyScript\rawdata\Wear\BUG'
file_name = "HeartRate_Static_TEST_20260129_214656_c133.rawdata"

# is_ring = True

file_path = os.path.join(directory, file_name)

real_path = os.path.realpath(file_path)
file_name = os.path.basename(file_path)
file_name_without_ext, ext = os.path.splitext(file_name)
directory = os.path.dirname(real_path)
info =  parse_filename(file_name)
mac = info.get('mac','0000').lower()

print(f"real_path: {real_path} {directory} {file_name_without_ext}{ext}  {mac} \n")

result = get_raw_data(real_path,is_ring=is_ring)

save_dir = os.path.join(directory, "data")
os.makedirs(save_dir, exist_ok=True)


data_type_dict = result.get('data_types', {})

base_info_df = result.get('base_info_df', pd.DataFrame(columns=['unix_timestamp']))
wear_info_df = result.get('wear_info_df', pd.DataFrame(columns=['unix_timestamp']))
tyhx_df = result.get('TYHX_data_df', pd.DataFrame(columns=['unix_timestamp']))
psp_fifo_df = result.get('psp_fifo_df', pd.DataFrame(columns=['unix_timestamp']))
if is_ring:
    TYHX_3697_data_df = result.get('TYHX_3697_data_df', pd.DataFrame(columns=['unix_timestamp']))
else:
    tyhx_agc_data_df = result.get('TYHX_agc_data_df', pd.DataFrame(columns=['unix_timestamp']))


ppg_g_df = result.get('ppg_g_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_r_df = result.get('ppg_r_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_ir_df = result.get('ppg_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_r_ir_df = result.get('ppg_r_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
acc_df =  result.get('acc_df', pd.DataFrame(columns=['unix_timestamp']))
gyro_df =  result.get('gyro_df', pd.DataFrame(columns=['unix_timestamp']))

sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 
print(f"文件 {file_path} 解析完成,包含数据类型: ")
print(f"{"\n".join(readable_types)}")

(combined_ppg_df, 
wear_info_df,
expanded_acc_df, 
expanded_ppg_g_df, 
expanded_ppg_ir_df, 
expanded_ppg_r_df)= process_and_merge_sensor_data(acc_df, ppg_g_df, ppg_ir_df,ppg_r_df, wear_info_df) 


expanded_ppg_r_ir_df = expand_ppg_data(ppg_r_ir_df, 'ppg_data')
expanded_ppg_r_ir_df = expanded_ppg_r_ir_df.rename(columns={'ppg_data':'ppg_value'})

if 'ppg_data_0' in ppg_g_df.columns:
    expanded_ppg_g1_df = expand_ppg_data(ppg_g_df, 'ppg_data_0')
    expanded_ppg_g1_df = expanded_ppg_g1_df.rename(columns={'ppg_data_0':'ppg_value'})
else:
    expanded_ppg_g1_df = pd.DataFrame()  
if 'ppg_data_1' in ppg_g_df.columns:
    expanded_ppg_g2_df = expand_ppg_data(ppg_g_df, 'ppg_data_1')
    expanded_ppg_g2_df = expanded_ppg_g2_df.rename(columns={'ppg_data_1':'ppg_value'})
else:
    expanded_ppg_g2_df = pd.DataFrame()


acc_df.to_csv(f"{save_dir}/{file_name_without_ext}_acc_data.csv", index=False, encoding='utf-8')
expanded_ppg_g_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_g.csv", index=False, encoding='utf-8')
expanded_ppg_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_ir.csv", index=False, encoding='utf-8')
tyhx_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_data.csv", index=False, encoding='utf-8')
psp_fifo_df.to_csv(f"{save_dir}/{file_name_without_ext}_psp_fifo_data.csv", index=False, encoding='utf-8')


if not is_ring:
    tyhx_agc_data_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_agc_data.csv", index=False, encoding='utf-8') # type: ignore

expanded_ppg_r_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r_ir.csv", index=False, encoding='utf-8')
expanded_ppg_r_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r.csv", index=False, encoding='utf-8')
# combined_ppg_df.to_csv(f"{save_dir}/{file_name_without_ext}_combined_ppg.csv", index=False, encoding='utf-8')




plot_multi_ppg_data(ppg_g_df=expanded_ppg_g1_df, 
                    ppg_g2_df=expanded_ppg_g2_df,
                    ppg_ir_df=expanded_ppg_ir_df, 
                    ppg_r_df=expanded_ppg_r_df, 
                    ppg_r_ir_df=expanded_ppg_r_ir_df,
                    acc_df=expanded_acc_df,
                    directory=save_dir, 
                    file_name_without_ext=file_name_without_ext,
                    is_show=True)
# plot_tyhx_currents(tyhx_df,
#                    directory=save_dir, 
#                    file_name_without_ext=file_name_without_ext,
#                    is_show=True)
# #  调用函数绘制结果
# plot_wear_detection_results(expanded_ppg_ir_df, 
#                             expanded_ppg_g_df, 
#                             expanded_ppg_r_df, 
#                             expanded_acc_df, 
#                             wear_info_df, 
#                             save_dir, 
#                             file_name_without_ext,
#                             is_show=True,result_type='original')

real_path: G:\PyScript\rawdata\Wear\BUG\HeartRate_Static_TEST_20260129_214656_c133.rawdata G:\PyScript\rawdata\Wear\BUG HeartRate_Static_TEST_20260129_214656_c133.rawdata  c133 

文件 G:\PyScript\rawdata\Wear\BUG\HeartRate_Static_TEST_20260129_214656_c133.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 106 
18  RAW_ACC               原始加速度                  count: 321 
19  RAW_GYRO              原始陀螺仪                  count: 321 
1a  RAW_GEOM              原始地磁                   count: 196 
69  MOTION_INFO           运动识别                   count: 107 
processed_ppg_g_df 为空或缺少 unix_timestamp 列
processed_ppg_ir_df 为空或缺少 unix_timestamp 列
processed_ppg_r_df 为空或缺少 unix_timestamp 列


g:\PyScript\script\common.py:823: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_ppg_df['green_size'] = combined_ppg_df['green_size'].fillna(0).astype(int)
g:\PyScript\script\common.py:824: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combined_ppg_df['ir_size'] = combined_ppg_df['ir_size'].fillna(0).astype(int)


图形已保存至: G:\PyScript\rawdata\Wear\BUG\data\HeartRate_Static_TEST_20260129_214656_c133_multi_ppg.png


In [ ]:
%matplotlib qt
plot_wear_detection_results(expanded_ppg_ir_df, 
                            expanded_ppg_g_df, 
                            expanded_ppg_r_df, 
                            expanded_acc_df, 
                            wear_info_df, 
                            save_dir, 
                            file_name_without_ext,
                            is_show=True,result_type='original')

图形已保存至: G:\PyScript\rawdata\Wear\CWR12G_0128_test\data/HeartRate_Static_murali_20260127_200804_5e7a_wear_original.png


In [71]:
def merge_sensor_packets(ppg_g_df, ppg_ir_df, acc_data_df, tolerance_sec=10):
    """
    针对 SensorHub 包级别的合并逻辑 (一秒一包，不展开)
    """
    # 1. 确保所有 DF 按 unix 排序
    # 即使是包匹配，merge_asof 也要求顺序一致
    ppg_g = ppg_g_df.sort_values('unix_timestamp').copy()
    ppg_ir = ppg_ir_df.sort_values('unix_timestamp').copy()
    acc = acc_data_df.sort_values('unix_timestamp').copy()

    # 2. PPG 内部匹配 (G 包匹配 IR 包)
    # 假设每秒可能产生重复包戳，依然建议保留 sub_id 逻辑防止笛卡尔积
    ppg_g['sub_id'] = ppg_g.groupby('unix_timestamp').cumcount()
    ppg_ir['sub_id'] = ppg_ir.groupby('unix_timestamp').cumcount()

    # 第一步：G 包与 IR 包一对一合并
    # 只要 unix_timestamp 和 序号对上就合并，对不上则 IR 列为 NaN
    ppg_combined = pd.merge(
        ppg_g, 
        ppg_ir, 
        on=['unix_timestamp', 'sub_id'], 
        how='left', 
        suffixes=('_g', '_ir')
    )

    # 3. 关联 ACC 包 (允许复用，带 10s 容差)
    # direction='nearest' 会寻找时间上最接近的那个 ACC 包
    final_df = pd.merge_asof(
        ppg_combined,
        acc,
        on='unix_timestamp',
        direction='nearest',
        tolerance=tolerance_sec,
        suffixes=('', '_acc')
    )

    # 移除辅助序号列
    return final_df.drop(columns=['sub_id'], errors='ignore')


def _ensure_ppg_array(x):
    if isinstance(x, np.ndarray):
        return x
    if isinstance(x, list):
        return np.asarray(x)
    return np.array([], dtype=float)

In [72]:
acc_data_copy = acc_df[['unix_timestamp','acc_data']].copy()
ppg_g_df_copy = ppg_g_df[['unix_timestamp','data','ppg_data_0']].copy()
ppg_g_df_copy['g_current'] = ppg_g_df_copy['data'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else 1)
ppg_ir_df_copy = ppg_ir_df[['unix_timestamp','ppg_data_0']].copy()

# acc_data_copy = process_timestamp_correction(acc_data_copy, tolerance=5)
# ppg_g_df_copy = process_timestamp_correction(ppg_g_df_copy, tolerance=5)
# ppg_ir_df_copy = process_timestamp_correction(ppg_ir_df_copy, tolerance=5)

merge_df = merge_sensor_packets(ppg_g_df_copy, ppg_ir_df_copy, acc_data_copy, tolerance_sec=10)
merge_df.rename(columns={
    'ppg_data_0_g': 'ppg_g_data',
    'ppg_data_0_ir': 'ppg_ir_data'
}, inplace=True)
merge_df.drop(columns=['data'], inplace=True)
# merge_df['ppg_g_data'].fillna(method='ffill', inplace=True)
merge_df['ppg_g_data'] = merge_df['ppg_g_data'].apply(_ensure_ppg_array)
merge_df['ppg_ir_data'] = merge_df['ppg_ir_data'].apply(_ensure_ppg_array)

In [76]:
ir_baseline = 0
print(f"使用的ir_baseline: {ir_baseline} for mac: {mac}")
wear_result = run_wear_detection_algorithm(merge_df,directory,file_name_without_ext,
                                            ir_threshold=20000,
                                            ir_baseline=ir_baseline)
wear_df = pd.DataFrame(wear_result)

使用的ir_baseline: 0 for mac: f246
alg_wear_new Ver: 0.2.14

Successfully processed 626 rows.
日志已保存至: G:\PyScript\rawdata\Wear\CHR03\BUG\HeartRate_Static_hlh_20260128_000150_f246_log.txt


In [ ]:
bursts, gaps = detect_jitter_patterns(df1, 'final_corrected_unix')
print("--- 1. 多包堆积检测 (Bursts) ---")
print(bursts.to_string(index=False))
print("\n--- 2. 时间空缺检测 (Gaps) ---")
print(gaps .to_string(index=False) if 'gap_df' in locals() else gaps.to_string(index=False))

In [45]:
acc_data_copy1['final_corrected_unix'].plot()
acc_data_copy1['unix_timestamp'].plot()

<Axes: >

In [ ]:



base_info_df = result.get('base_info_df', None)
ppg_g_df = result.get('ppg_g_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_ir_df = result.get('ppg_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
ppg_r_df = result.get('ppg_r_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
wear_info_df = result.get('wear_info_df', None)
acc_df =  result.get('acc_df', None)
data_type_dict = result.get('data_types', {})


sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 







文件 G:\PyScript\rawdata\Wear\CHR03\HeartRate_Static_TEST_20260128_180423_f246.rawdata 解析完成,包含数据类型: 
00  BASE_INFO             基础信息                   count: 63 \n15  RAW_PPG_GREEN         原始PPG绿光                count: 61 \n17  RAW_PPG_IR            原始PPG红外                count: 49 \n18  RAW_ACC               原始加速度                  count: 126 \n21  RAW_ACC_TO_PPG        原始加速度到PPG              count: 122 \n33  AMBIENT_INFO          环境信息                   count: 61 \n67  SLEEP_INFO            睡眠信息                   count: 63 \n68  HRV_INFO              心率变异性信息                count: 61 \n69  MOTION_INFO           运动识别                   count: 63 \n71  WEAR_INFO             佩戴信息                   count: 62 \n78  HEART_ACC_INFO        心率加速度信息                count: 61 \n82  TYHX_AGC_DATA         TYHX AGC数据             count: 62 \n91  GREEN_PSP_INFO        绿光PSP信息                count: 122 \n96  TIME_PSP_INFO         时间PSP信息                count: 61 \n99  CREEK_HR_INFO         CREEK心率信息         

In [ ]:
df = wear_info_df[wear_info_df['unix_timestamp'] <= 1768590022 + 100].copy()

import matplotlib.pyplot as plt

# 绘制 df['wear_flag']
plt.figure(figsize=(15, 4))
plt.plot(df['unix_timestamp'], df['wear_flag'], marker='o', markersize=3, linewidth=1, label='Wear Flag', color='orange')

# 标注 1768590022 的位置
plt.axvline(x=1768590022, color='red', linestyle='--', linewidth=2, label='Timestamp 1768590022')

plt.title('Wear Flag with Annotation at 1768590022', fontsize=14, fontweight='bold')
plt.xlabel('Unix Timestamp (s)', fontsize=12)
plt.ylabel('Wear Flag', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
%matplotlib inline  

directory = r'D:\RawData'

# 获取目录下的所有文件
all_files = os.listdir(directory)

# 过滤出 .rawdata 文件
rawdata_files = [f for f in all_files if f.endswith('.rawdata')]

for file_name in rawdata_files[:]:
    file_path = os.path.join(directory, file_name)

    real_path = os.path.realpath(file_path)
    file_name = os.path.basename(file_path)
    file_name_without_ext, ext = os.path.splitext(file_name)
    directory = os.path.dirname(real_path)
    info =  parse_filename(file_name)
    mac = info.get('mac','0000').lower()
    save_dir = os.path.join(directory, "data")
    os.makedirs(save_dir, exist_ok=True)

    print(f"real_path: {real_path} {directory} {file_name_without_ext}{ext}  {mac} \n")

    result = get_raw_data(real_path)


    base_info_df = result.get('base_info_df', None)
    ppg_g_df = result.get('ppg_g_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
    ppg_r_df = result.get('ppg_r_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
    ppg_ir_df = result.get('ppg_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
    ppg_r_ir_df = result.get('ppg_r_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
    wear_info_df = result.get('wear_info_df', None)
    acc_df =  result.get('acc_df', pd.DataFrame(columns=["unix_timestamp","arr_size","acc_data"]))
    data_type_dict = result.get('data_types', {})
    tyhx_df = result.get('TYHX_data_df', pd.DataFrame())
    tyhx_agc_data_df = result.get('TYHX_agc_data_df', pd.DataFrame())

    
    sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
    readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 


    print(f"文件 {file_path} 解析完成,包含数据类型: ")
    print(f"{"\n".join(readable_types)}")






    (combined_ppg_df, 
    wear_info_df,
    expanded_acc_df, 
    expanded_ppg_g_df, 
    expanded_ppg_ir_df, 
    expanded_ppg_r_df)= process_and_merge_sensor_data(acc_df, ppg_g_df, ppg_ir_df,ppg_r_df, wear_info_df) 

    expanded_ppg_r_ir_df = expand_ppg_data(ppg_r_ir_df, 'ppg_data')
    expanded_ppg_r_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r_ir.csv", index=False, encoding='utf-8')
    combined_ppg_df.to_csv(f"{save_dir}/{file_name_without_ext}_combined_ppg.csv", index=False, encoding='utf-8')
    expanded_ppg_g_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_g.csv", index=False, encoding='utf-8')
    expanded_ppg_r_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_r.csv", index=False, encoding='utf-8')
    expanded_ppg_ir_df.to_csv(f"{save_dir}/{file_name_without_ext}_expanded_ppg_ir.csv", index=False, encoding='utf-8')
    tyhx_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_data.csv", index=False, encoding='utf-8')
    tyhx_agc_data_df.to_csv(f"{save_dir}/{file_name_without_ext}_tyhx_agc_data.csv", index=False, encoding='utf-8')
    

    # ppg_g_25hz = resample_to_25hz(
    #     expanded_ppg_g_df, 
    #     value_columns=['ppg_value'], 
    #     data_name="PPG Green"
    # )

    # # 2. 降采样加速度数据到25Hz  
    # acc_25hz = resample_to_25hz(
    #     expanded_acc_df,
    #     value_columns=['x', 'y', 'z'],
    #     data_name="ACC"
    # )

    # # 3. 合并ACC和PPG数据
    # merged_data = merge_acc_ppg_data(acc_25hz, ppg_g_25hz, tolerance_ms=20)
    # merged_data.to_csv(f"{save_dir}/{file_name_without_ext}_merged_ppg_acc25hz.csv", index=False, encoding='utf-8')
    # 调用函数绘制结果
    plot_wear_detection_results(expanded_ppg_ir_df, 
                                expanded_ppg_g_df, 
                                expanded_ppg_r_df, 
                                expanded_acc_df, 
                                wear_info_df, 
                                save_dir, 
                                file_name_without_ext,
                                is_show=False,result_type='original')


In [ ]:
# 降采样数据  -> 25Hz

directory = r'D:\FeishuDownload\0108'
directory = r'G:\PyScript\rawdata\HeartRate\数据文件\运动心率'
directory = r'G:\PyScript\rawdata\HeartRate\数据文件\运动心率\MAE过大'
# 获取目录下的所有文件
all_files = os.listdir(directory)
rawdata_files = [f for f in all_files if f.endswith('.rawdata')]
# file_name = rawdata_files[0]

for file_name in rawdata_files[:]:

    file_path = os.path.join(directory, file_name)

    real_path = os.path.realpath(file_path)
    file_name = os.path.basename(file_path)
    file_name_without_ext, ext = os.path.splitext(file_name)
    directory = os.path.dirname(real_path)
    info =  parse_filename(file_name)
    mac = info.get('mac','0000').lower()
    save_dir = os.path.join(directory, "data")
    os.makedirs(save_dir, exist_ok=True)

    print(f"real_path: {real_path} {directory} {file_name_without_ext}{ext}  {mac} \n")

    result = get_raw_data(real_path)


    data_type_dict = result.get('data_types', {})

    base_info_df = result.get('base_info_df', pd.DataFrame(columns=['unix_timestamp']))
    wear_info_df = result.get('wear_info_df', pd.DataFrame(columns=['unix_timestamp']))
    tyhx_df = result.get('TYHX_data_df', pd.DataFrame(columns=['unix_timestamp']))
    tyhx_agc_data_df = result.get('TYHX_agc_data_df', pd.DataFrame(columns=['unix_timestamp']))

    ppg_g_df = result.get('ppg_g_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
    ppg_r_df = result.get('ppg_r_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
    ppg_ir_df = result.get('ppg_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
    ppg_r_ir_df = result.get('ppg_r_ir_df',pd.DataFrame(columns=['unix_timestamp','data','ppg_data']))
    acc_df =  result.get('acc_df', pd.DataFrame(columns=['unix_timestamp']))


    sorted_data_types = sorted(data_type_dict.keys(), key=lambda x: int(x, 16))
    readable_types = [f"{dt}  {get_full_data_type(dt)}  count: {data_type_dict[dt]} " for dt in sorted_data_types] 


    print(f"文件 {file_path} 解析完成,包含数据类型: ")
    print(f"{"\n".join(readable_types)}")
    value_columns ='ppg_data_0'
    expanded_ppg_g_df = expand_ppg_data(ppg_g_df, value_columns)
    expanded_acc_df = expand_acc_data(acc_df)
    ppg_g_25hz = resample_to_25hz(
        expanded_ppg_g_df, 
        value_columns=[value_columns], 
        data_name="PPG Green"
    )

    
    acc_25hz = resample_to_25hz(
        expanded_acc_df,
        value_columns=['x', 'y', 'z'],
        data_name="ACC"
    )
    df1 = base_info_df[['unix_timestamp','label_hr']]
    merged_data_df = merge_acc_ppg_data(acc_25hz, ppg_g_25hz, tolerance_ms=20)
    merged_data_df['unix_timestamp_floor'] = merged_data_df['unix_timestamp'].apply(lambda x: int(x))
    merged_data_df = pd.merge(merged_data_df, df1, 
                  left_on='unix_timestamp_floor', 
                  right_on='unix_timestamp', 
                  how='left',
                  suffixes=('', '_hr'))
    merged_data_df = merged_data_df. drop(['unix_timestamp_floor','unix_timestamp_hr'], axis=1)
    merged_data_df['label_hr'] = merged_data_df['label_hr'].fillna(0)
    merged_data_df['label_hr'] = merged_data_df['label_hr'].apply(lambda x: int(x))

    numeric_columns = ['x', 'y', 'z', 'ppg_data_0']
    for col in numeric_columns:
        if col in merged_data_df.columns:
            merged_data_df[col] = merged_data_df[col].round(1)


    merged_data_df.to_csv(f"{save_dir}/{file_name_without_ext}_merged_ppg_acc25hz.csv", index=False, encoding='utf-8')

In [ ]:
merged_data_df[ merged_data_df['label_hr']==0 ]

In [ ]:
merged_data.head(10).to_clipboard()

In [ ]:
# 去除重复的 unix_timestamp
print(f"base_info_df 去重前: {len(base_info_df)} 行, 重复: {base_info_df['unix_timestamp'].duplicated().sum()} 条")
base_info_df = base_info_df.drop_duplicates(subset=['unix_timestamp'], keep='first')
print(f"base_info_df 去重后: {len(base_info_df)} 行\n")

print(f"tyhx_df 去重前: {len(tyhx_df)} 行, 重复: {tyhx_df['unix_timestamp'].duplicated().sum()} 条")
tyhx_df = tyhx_df.drop_duplicates(subset=['unix_timestamp'], keep='first')
print(f"tyhx_df 去重后: {len(tyhx_df)} 行\n")

print(f"tyhx_agc_data_df 去重前: {len(tyhx_agc_data_df)} 行, 重复: {tyhx_agc_data_df['unix_timestamp'].duplicated().sum()} 条")
tyhx_agc_data_df = tyhx_agc_data_df.drop_duplicates(subset=['unix_timestamp'], keep='first')
print(f"tyhx_agc_data_df 去重后: {len(tyhx_agc_data_df)} 行\n")

print(f"wear_info_df 去重前: {len(wear_info_df)} 行, 重复: {wear_info_df['unix_timestamp'].duplicated().sum()} 条")
wear_info_df = wear_info_df.drop_duplicates(subset=['unix_timestamp'], keep='first')
print(f"wear_info_df 去重后: {len(wear_info_df)} 行")


In [ ]:
ppg_g_df

In [ ]:
import pandas as pd
import numpy as np

def unwrap_with_segment_reset(df, nominal_period, sn_col='serial_number', ts_col='unix_timestamp'):
    """
    当 SN 不连续时,开启新的片段,并重新校准时间基准。
    
    参数:
        df: 输入的 DataFrame
        nominal_period: 理论采样周期 (如 100Hz 对应 0.01)
        sn_col: 原始 uint8 序列号列
        ts_col: 原始 unix 时间戳列
    """
    if df is None or df.empty:
        return df

    df = df.copy()
    
    # 1. 识别断点 (只有当 SN 恰好递增 1 时才算连续,考虑 255->0)
    # 计算 (curr - prev) % 256
    sn_values = df[sn_col].values.astype(np.int64)
    sn_diff = np.diff(sn_values) % 256
    
    # 连续的定义是 diff == 1。如果不等于 1,说明有丢包、重复或设备重启。
    # 第一个点强制设为断点开始
    is_break = np.concatenate(([True], sn_diff != 1))
    
    # 2. 生成片段 ID (每一个连续段分配一个唯一的 ID)
    df['segment_id'] = np.cumsum(is_break)
    
    # 3. 片段内处理
    # 我们需要计算每个点在自己片段内的偏移量 (0, 1, 2...)
    df['step_in_segment'] = df.groupby('segment_id').cumcount()
    
    # 4. 重构时间戳
    # 每个片段的第一个时间戳作为基准点
    segment_starts = df.groupby('segment_id')[ts_col].transform('first')
    df['reconstructed_ts'] = segment_starts + df['step_in_segment'] * nominal_period
    
    # 5. 生成全局连续的序列号 (单纯按接收顺序累加)
    df['global_sn'] = np.arange(len(df))
    
    # 打印分析报告
    num_segments = df['segment_id'].max()
    print(f"--- 序列分析报告 ---")
    print(f"总数据量: {len(df)} 行")
    print(f"检测到连续片段数: {num_segments}")
    print(f"平均片段长度: {len(df)/num_segments:.2f}")
    
    return df

# --- 使用示例 ---
# df = pd.read_csv("your_data.csv")
# # 假设频率是 50Hz
# df_clean = unwrap_with_segment_reset(df, nominal_period=0.02)

In [ ]:
# 使用函数进行数据处理
# 1. 降采样PPG Green数据到25Hz
ppg_g_25hz = resample_to_25hz(
    expanded_ppg_g_df, 
    value_columns=['ppg_value'], 
    data_name="PPG Green"
)

# 2. 降采样加速度数据到25Hz  
acc_25hz = resample_to_25hz(
    expanded_acc_df,
    value_columns=['x', 'y', 'z'],
    data_name="ACC"
)

# 3. 合并ACC和PPG数据
merged_data = merge_acc_ppg_data(acc_25hz, ppg_g_25hz, tolerance_ms=20)

# 显示合并后的数据预览
if merged_data is not None and not merged_data.empty:
    print("\n合并后数据预览:")
    print(merged_data.head())

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(20, 16), sharex=True)

# 绘制 PPG Green
if expanded_ppg_g_df is not None and not expanded_ppg_g_df.empty:
    axes[0].plot(expanded_ppg_g_df['unix_timestamp'], expanded_ppg_g_df['ppg_value'], 
                 label='PPG Green', color='green', linewidth=1)
    axes[0].set_ylabel('PPG Green Value')
    axes[0].set_title('PPG Green Data')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

# 绘制 PPG Red
if expanded_ppg_r_df is not None and not expanded_ppg_r_df.empty:
    axes[1].plot(expanded_ppg_r_df['unix_timestamp'], expanded_ppg_r_df['ppg_value'], 
                 label='PPG Red', color='red', linewidth=1)
    axes[1].set_ylabel('PPG Red Value')
    axes[1].set_title('PPG Red Data')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

# 绘制 PPG R_IR
if expanded_ppg_r_ir_df is not None and not expanded_ppg_r_ir_df.empty:
    axes[2].plot(expanded_ppg_r_ir_df['unix_timestamp'], expanded_ppg_r_ir_df['ppg_value'], 
                 label='PPG R_IR', color='purple', linewidth=1)
    axes[2].set_ylabel('PPG R_IR Value')
    axes[2].set_title('PPG R_IR Data')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)


# 绘制 PPG IR
if expanded_ppg_ir_df is not None and not expanded_ppg_ir_df.empty:
    axes[3].plot(expanded_ppg_ir_df['unix_timestamp'], expanded_ppg_ir_df['ppg_value'], 
                 label='PPG IR', color='pink', linewidth=1)
    axes[3].set_ylabel('PPG IR Value')
    axes[3].set_title('PPG IR Data')
    axes[3].set_xlabel('Unix Timestamp (s)')
    axes[3].legend()
    axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
"Wear_Unwear_shb_20251219_175814_d9e5.rawdata"

In [ ]:
directory = r'G:\python\rawdata_Wearing'
# Sleep_Normal_chenyx_20260104_235014_109a.pkl

In [ ]:
import pickle

file_path = r'G:\python\rawdata_Wearing\Sleep_Normal_chenyx_20260104_235014_109a.pkl'

with open(file_path, 'rb') as f:
    data = pickle.load(f)

print("Loaded data from pickle file:")
print(data)